# Tensorflow

## 1. 系统架构

![](/images/arch.png)



最下层是网络通信层和设备管理层。网络通信层包括gRPC（google Remote Procedure Call Protocol）和远程直接数据存取（Remote Direct Memory Access，RDMA），这都是在分布式计算时需要用到的,

设备管理层包括 TensorFlow 分别在 CPU、GPU、FPGA 等设备上的实现，也就是对上层提供了一个统一的接口，使上层只需要处理卷积等逻辑，而不需要关心在硬件上的卷积的实现过程.

数据操作层，主要包括卷积函数、激活函数等操作

图计算层，包含本地计算图和分布式计算图的实现

## 2. 设计理念

TensorFlow 将图的定义和图的运行完全分开，是一种符号式编程，符号式编程涉及很多的嵌入和优化，不容易理解和调试，但运行速度相对有所提升。符号式计算一般是先定义各种变量，然后建立一个数据流图，在数据流图中规定各个变量之间的计算关系，最后需要对数据流图进行编译，但此时的数据流图还是一个空壳儿，里面没有任何实际数据，只有把需要运算的输入放进去后，才能在整个模型中形成数据流，从而形成输出值。

TensorFlow 中涉及的运算都要放在图中，而图的运行只发生在会话（session）中。开启会话后，就可以用数据去填充节点，进行运算；关闭会话后，就不能进行计算了。因此，会话提供了操作运行和 Tensor 求值的环境。

## 编程模型
TensorFlow 是用数据流图做计算的，因此我们先创建一个数据流图（也称为网络结构图）,下图是一个是一个简单的回归模型的数据流图，

![](/images/dataflow.png)

TensorFlow 是指“张量的流动”,由上图可以看出，TensorFlow 的数据流图是由节点（node）和边（edge）组成的有向无环图,TensorFlow 由 Tensor 和 Flow 两部
分组成，Tensor（张量）代表了数据流图中的边，而 Flow（流动）这个动作就代表了数据流图中节点所做的操作。

#### 边
TensorFlow 的边有两种连接关系：数据依赖(实线)和控制依赖(虚线),用的较多的是数据依赖边，代表数据，即张量，张量在数据流图中从前往后流动一遍就完成了一次前向传播（forword propagation），而残差②从后向前流动一遍就完成了一次反向传播（backword propagation）

#### 节点
又称为算子，是一个对数据的操作,也可以表示数据输入（feed in）的起点以及输出（push out）的终点，或者是读取/写入持久变量（persistent variable）的终点.

#### 图和会话
我们知道，tensorflow最重要的步骤便是 构造图 和 计算图，那么如何构造图， 计算图呢?

In [2]:
# 建图的第一步是创建各个节点

import tensorflow as tf

# 创建一个常量运算操作，产生一个 1×2 矩阵 
matrix1 = tf.constant([[3., 3.]]) 

# 创建另外一个常量运算操作，产生一个 2×1 矩阵 
matrix2 = tf.constant([[2.],[2.]]) 

# 创建一个矩阵乘法运算 ，把matrix1 和matrix2 作为输入 
# 返回值product 代表矩阵乘法的结果 
product = tf.matmul(matrix1, matrix2) 

# 启动图的第一步是创建一个Session 对象,会话（session）提供在图中执行操作的一些方法
with tf.Session() as sess:
    res = sess.run([product]) # 调用 Session 对象的run()方法来执行图
    print(res) 

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


[array([[12.]], dtype=float32)]


会话是图交互的一个桥梁，一个会话可以有多个图，会话可以修改图的结构，也可以往图中注入数据进行计算。因此，会话主要有两个API 接口：Extend 和 Run。Extend 操作是在Graph中添加节点和边，Run 操作是输入计算的节点和填充必要的数据后，进行运算，并输出运算结果

#### 常量和变量

TensorFlow 中最基本的单位是常量（Constant）、变量（Variable）和占位符（Placeholder）。常量定义后值和维度不可变，变量定义后值可变而维度不可变。在神经网络中，变量一般可作为储存权重和其他信息的矩阵，而常量可作为储存超参数或其他结构信息的变量。

In [3]:
# 创建一个变量，初始化为标量 0
state = tf.Variable(0, name='c')

# 创建一个常量
input1 = tf.constant(3.0)

TensorFlow 还提供了填充机制，可以在构建图时使用 tf.placeholder()临时替代任意操作的张量，在调用Session 对象的run()方法去执行图时，使用填充数据作为调用的参数，调用结束后，填充数据就消失。

In [5]:
input1 = tf.placeholder(tf.float32)
input2 = tf.placeholder(tf.float32)
out = tf.multiply(input1, input2)
with tf.Session() as sess:
    print(sess.run([out], feed_dict={input1: [2.], input2: [3.]}))

[array([6.], dtype=float32)]


#### 设备
设备（device）是指一块可以用来运算并且拥有自己的地址空间的硬件，如GPU 和 CPU.

TensorFlow 为了实现分布式执行操作，充分利用计算资源，可以明确指定操作在哪个设备上执行


In [8]:
with tf.Session() as sess: 
    # 指定在第二个gpu 上运行 
    with tf.device("/gpu:1"): 
        matrix1 = tf.constant([[3., 3.]]) 
        matrix2 = tf.constant([[2.],[2.]]) 
        product = tf.matmul(matrix1, matrix2) 

#### 命名空间
主要是因为 变量共享 的需求。而这就不得不谈到tf. get_variable()了。因为如果使用Variable 的话每次都会新建变量，但是大多数时候我们是希望一些变量重用的，所以就用到了get_variable()。它会去搜索变量名，然后没有就新建，有就直接用。既然用到变量名了，就涉及到了名字域的概念。"通过不同的域来区别变量名"，毕竟让我们给所有变量都直接取不同名字还是有点辛苦的。所以为什么会有scope 的概念。name_scope 作用于操作，variable_scope 可以通过设置reuse 标志以及初始化方式来影响域下的变量。
当然对我们而言还有个更直观的感受就是：在tensorboard 里可视化的时候用名字域进行封装后会更清晰

> 作者：ycszen
链接：https://www.zhihu.com/question/54513728/answer/177901159



在 TensorFlow 中有两个作用域（scope），一个是name_scope，另一个是variable_scope。它们究竟有什么区别呢？简而言之，name_scope主要是给variable_name 加前缀，也可以给op_name加前缀；name_scope 是给op_name 加前缀

##### variable_scope
variable_scope 主要用在循环神经网络（RNN）的操作中，其中需要大量的共享变量,

变量作用域可以默认携带一个初始化器，在这个作用域中的子作用域或变量都可以继承或者重写父作用域初始化器中的值

In [ ]:
# variable_scope 变量作用域机制在 TensorFlow 中主要由两部分组成:
#v = tf.get_variable(name, shape, dtype, initializer) # 通过所给的名字创建或是返回一个变量 
#tf.variable_scope(<scope_name>) # 为变量指定命名空间 
#当 tf.get_variable_scope().reuse == False 时，variable_scope 作用域只能用来创建新变量
with tf.variable_scope("foo"): 
    v = tf.get_variable("v", [1]) 
    #v2 = tf.get_variable("v", [1]) 
#assert v.name == "foo/v:0" 
# Error 因为 v 这个变量已经被定义过了, 而reuse 默认为 False

# 当 tf.get_variable_scope().reuse == True 时，作用域可以共享变量
with tf.variable_scope("foo", reuse=True): 
    #也可以写成： 
    #scope.reuse_variables() 
    v1 = tf.get_variable("v", [1]) 
#assert v1 == v 

##### name_scope
TensorFlow 中常常会有数以千计的节点，在可视化的过程中很难一下子展示出来，因此用 name_scope 为变量划分范围，在可视化中，这表示在计算图中的一个层级。name_scope会影响 op_name，不会影响用 get_variable()创建的变量，而会影响通过 Variable()创建的变量。

name_scope 对用get_variable()创建的变量的名字不会有任何影响，而 Variable()创建的操作会被加上前缀，并且会给操作加上名字前缀。

In [ ]:
with tf.variable_scope("foo"): 
    with tf.name_scope("bar"): 
        v = tf.get_variable("v", [1]) 
        b = tf.Variable(tf.zeros([1]), name='b') 
        x = 1.0 + v 
assert v.name == "foo/v:0" 
assert b.name == "foo/bar/b:0" 
assert x.op.name == "foo/bar/add" 

#### 批标准化
批标准化（batch normalization，BN）是为了克服神经网络层数加深导致难以训练而诞生的。我们知道，深度神经网络随着网络深度加深，训练起来会越来越困难，收敛速度会很慢，常常会导致梯度弥散问题（vanishing gradient problem）。

批标准化一般用在非线性映射（激活函数）之前，对 x=Wu+b 做规范化，使结果（输出信号各个维度）的均值为 0，方差为 1。让每一层的输入有一个稳定的分布会有利于网络的训练。 

##### 优点 
批标准化通过规范化让激活函数分布在线性区间，结果就是加大了梯度，让模型更加大胆地进行梯度下降，于是有如下优点： 
1. 加大探索的步长，加快收敛的速度；
2. 更容易跳出局部最小值；
3. 破坏原来的数据分布，一定程度上缓解过拟合。
因此，在遇到神经网络收敛速度很慢或梯度爆炸（gradient explode）等无法训练的情况下，都可以尝试用批标准化来解决

In [ ]:
# 这个步骤放在激活函数之前
# 计算Wx_plus_b的均值和方差，其中axes=[0]表示想要标准化的维度 
fc_mean, fc_var = tf.nn.moments(Wx_plus_b, axes=[0], ) 
scale = tf.Variable(tf.ones([out_size])) 
shift = tf.Variable(tf.zeros([out_size])) 
epsilon = 0.001 
Wx_plus_b = tf.nn.batch_normalization(Wx_plus_b, fc_mean, fc_var, shift, 
scale, epsilon) 
# 也就是在做： 
# Wx_plus_b = (Wx_plus_b - fc_mean) / tf.sqrt(fc_var + 0.001) 
# Wx_plus_b = Wx_plus_b * scale + shift 

### 神经元函数

#### 激活函数

激活神经网络中某一部分神经元，将激活信息向后传入下一层的神经网络。神经网络之所以能解决非线性问题（如语音、图像识别），本质上就是激活函数加入了非线性因素，弥补了线性模型的表达力，把“激活的神经元的特征”通过函数保留并映射到下一层。
常见的激活函数有 sigmoid、tanh、relu 和 softplus
1. sigmoid: tf.nn.sigmoid()
    * 优点: 它的输出映射在(0,1)内，单调连续，非常适合用作输出层，并且求导比较容易
    * 缺点: 一旦输入落入饱和区，f'(x)就会变得接近于 0，很容易产生梯度消失

2. tanh: tf.nn.tanh() 
    * 优点: 因为它的输出以 0 为中心，收敛速度比 sigmoid 要很
    * 缺点: 也具有饱和性，很容易产生梯度消失
    
3. relu: tf.nn.relu(),目前最受欢迎的激活函数
    * 优点: 缓解梯度消失问题，还能够更很地收敛，并提供了神经网络的稀疏表达能力
    * 缺点: 着训练的进行，部分输入会落到硬饱和区，导致对应的权重无法更新，称为“神经元死亡”
    
软饱和: 是指激活函数h(x)在取值趋于无穷大时，它的一阶导数趋于0。硬饱和是指当|x| > c 时，其中c 为常数，f'(x)=0。
relu 就是一类左侧硬饱和激活函数。 

###### 激活函数的选择 
大部分的卷积神经网络都采用 relu 作为激活函数，大概有 85%～90%的神经网络会采用 ReLU，10%～15%的神经网络会采用 tanh，尤其用在自然语言处理上

#### 卷积函数 
卷积函数是构建神经网络的重要支架，是在一批图像上扫描的二维过滤器。

tesorflow中的卷积函数主要有以下:

1. tf.nn.convolution(input, filter, padding, strides=None, dilation_rate=None, name=None, data_format =None)这个函数计算 N 维卷积的和。

    padding: 有两个值‘SAME’和‘VALID‘，前者做了适当的边缘填充使得卷积后图像尺寸不变；后者尺寸变化


2. tf.nn.conv2d(input, filter, strides, padding, use_cudnn_on_gpu=None, data_format=None,name=None)这个函数的作用是对一个四维的输入数据 input 和四维的卷积核filter 进行操作，然后对输入数据进行一个二维的卷积操作，最后得到卷积之后的结果。

3. tf.nn.depthwise_conv2d (input, filter, strides, padding, rate=None, name=None,data_format=None)，input.shape=[batch, in_height, in_width, in_channels], filter.shape=[filter_height, filter_width, in_channels, channel_multiplier],
channel_multiplier是指输出通道数，也就是你要输出的通道数，也就是你要使用的卷积核数,一般对于三通道图像做卷积，都是先加权求和再做卷积,形象化描述就是我先把3通道压扁成1通道，在把它用x个卷积核提溜成x通道,而depthwise_conv2d就不加权求和了，直接卷积，所以最后输出通道的总数是in_channels*channel_multiplier.

4. tf.nn.separable_conv2d(input, depthwise_filter, pointwise_filter, strides, padding, rate=None, name=None, data_format=None)是利用几个分离的卷积核去做卷积,将应用一个二维的卷积核，在每个通道上，以深度 channel_multiplier 进行卷积.

    depthwise_filter。一个张量，数据维度是四维[filter_height,filter_width,in_channels,channel_multiplier]卷积深度是1,
    
    pointwise_filter。一个张量，数据维度是四维[1,1,in_channels*channel_multiplier,out_channel]
    
    separable_conv2d是利用几个分离的卷积核去做卷积。首先用depthwise_filter做卷积，效果与depthwise_conv2d相同，然后用1x1的卷积pointwise_filter去做卷积,pointwise原理很简单,先把in_channels*channel_multiplier个通道压扁成1个通道，再用pointwise_filter这个1*1的卷积核提溜成out_channel个通道，所以pointwise_filter相当于out_channel个scalar。

#### 池化函数
在神经网络中，池化函数一般跟在卷积函数的下一层,池化操作是利用一个矩阵窗口在张量上进行扫描，将每个矩阵窗口中的值通过取最大值或平均值来减少元素个数。每个池化操作的矩阵窗口大小是由ksize 指定的，并且根据步长 strides决定移动步长。

常用的池化函数有:
1. tf.nn.avg_pool(value, ksize, strides, padding, data_format='NHWC', name=None)计算池化区域中元素的平均值

    value：一个四维的张量。数据维度是[batch, height, width, channels] 
    
    ksize：一个长度不小于4 的整型数组。每一位上的值对应于输入数据张量中每一维的窗口对应值
    
    strides：一个长度不小于4 的整型数组。该参数指定滑动窗口在输入数据张量每一维上的步长 
    
    padding：一个字符串，取值为SAME或者VALID,padding = 'SAME' 时，输出并不一定和原图size一致，但会保证覆盖原图所有像素，不会舍弃边上的莫些元素;
    
    shape(output) = (shape(value) - ksize + 1) / 计算池化区域中元素的最大值
2. tf.nn.max_pool(value, ksize, strides, padding, data_format='NHWC', name=None)计算池化区域中元素的最大值

#### 分类函数
1. tf.nn.sigmoid_cross_entropy_with_logits(logits, targets, name=None)

    最后一层不需要进行sigmoid 运算，此函数内部进行了sigmoid 操作
    
    输入：logits:[batch_size, num_classes],targets:[batch_size, size].logits 用最后一层的输入即可
    
    输出：loss [batch_size, num_classes] 
2. tf.nn.softmax(logits, dim=-1, name=None)

    计算Softmax 激活，也就是softmax = exp(logits) / reduce_sum(exp(logits), dim)
    
3. tf.nn.log_softmax(logits, dim=-1, name=None)

    计算log softmax 激活，也就是logsoftmax = logits - log(reduce_sum(exp(logits), dim))
    
4. tf.nn.softmax_cross_entropy_with_logits(_sentinel=None, labels=None, logits=None, dim=-1, name =None)

    输入：logits and labels 均为[batch_size, num_classes] 
    
    输出： loss:[batch_size], 里面保存的是batch 中每个样本的交叉熵 
    
5. tf.nn.sparse_softmax_cross_entropy_with_logits(logits, labels, name=None)

#### 优化方法
TensorFlow提供了很多的优化方法，主要有以下几个:
   * class tf.train.GradientDescentOptimizer 
   * class tf.train.AdadeltaOptimizer 
   * class tf.train.AdagradOptimizer 
   * class tf.train.AdagradDAOptimizer 
   * class tf.train.MomentumOptimizer 
   * class tf.train.AdamOptimizer 
   * class tf.train.FtrlOptimizer 
   * class tf.train.RMSPropOptimizer 
    
原理在ch1中有。应为他们都继承自Optimizer，他们的方法是通用的，只不过初始化参数有些不同,下面实施一些通用的方法

![](/images/optimizer.png)

> [详细参考](https://blog.csdn.net/shenxiaoming77/article/details/77169756)

#### 模型的保存与恢复

> [参考](https://cv-tricks.com/tensorflow-tutorial/save-restore-tensorflow-models-quick-complete-tutorial/)

# 使用MNIST数据集 实现 AlexNet 

In [ ]:
## 1.加载数据 以及 定义模型的一参数、模型所用的网络的参数以及数据的输入

import tensorflow as tf

# 导入数据
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data('/datasets/mnist', one_hot=True)

# 定义网络的超参数
learning_rate = .001
training_iters = 20000
batch_size = 128
display_step = 20

# 定义网络的参数
n_inputs = 784 # 28*28
n_classes = 10
dropout = .75

# 占位符
x = tf.placeholder(tf.float32, [None, n_inputs])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32)


## 2. 构建网络模型

# 定义卷积操作
def conv2d(name, x, W, b, strides=1, keep_size=True):
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x, name=name)

# 定义池化操作
def maxpool2d(name, x, k=2):
    return tf.nn.max_pool(x, ksize=[1,k,k,1], strides=[1,k,k,1], padding='SAME', name=name)

# 定义规范化操作
def norm(name, l_input, l_size=4):
    #  local response normalization--局部响应标准化。
    # LRN的作用就是，对位置(x, y)处的像素计算其与几个相邻的kernel maps的像素值的和，并除以这个和来归一化
    return tf.nn.lrn(l_input, l_size, bias=1.0, alpha=.001/9.0, beta=.75, name=name)

# 定义所有的网络参数
weights = {
    'wc1': tf.Variable(tf.random.normal([11, 11, 1, 96])),
    'wc1': tf.Variable(tf.random.normal([5, 5, 96, 256])),
    'wc1': tf.Variable(tf.random.normal([3, 3, 256, 384])),
    'wc1': tf.Variable(tf.random.normal([3, 3, 384, 384])),
    'wc1': tf.Variable(tf.random.normal([3, 3, 384, 256])),
    'wd1': tf.Variable(tf.random.normal([4*4*256, 4096])),
    'wd2': tf.Variable(tf.random_normal([4096, 4096])), 
    'out': tf.Variable(tf.random_normal([4096, 10])) 
}
biases = { 
    'bc1': tf.Variable(tf.random_normal([96])), 
    'bc2': tf.Variable(tf.random_normal([256])), 
    'bc3': tf.Variable(tf.random_normal([384])), 
    'bc4': tf.Variable(tf.random_normal([384])), 
    'bc5': tf.Variable(tf.random_normal([256])),  
    'bd1': tf.Variable(tf.random_normal([4096])), 
    'bd2': tf.Variable(tf.random_normal([4096])), 
    'out': tf.Variable(tf.random_normal([n_classes])) 
} 

# 定义 AlexNet 的网络模型
def alex_net(x, weights, biases, dropout):
    # reshape image vector
    x = tf.reshape(x, [-1, 28, 28, 1])
    
    # 第一层卷积
    conv1 = conv2d('conv1', x, weights['wc1'], biases['bc1'])
    # 下采样
    pool1 = maxpool2d('conv1', conv1)
    # 规范化
    norm1 = norm('norm1', pool1)
    
    
    # 第二层卷积
    conv2 = conv2d('conv2', x, weights['wc2'], biases['bc2'])
    # 下采样
    pool2 = maxpool2d('pool2', conv2)
    # 规范化
    norm2 = norm('norm2', pool2)
    
    # 第三层卷积
    conv3 = conv2d('conv3', x, weights['wc3'], biases['bc3'])
    # 下采样
    pool3 = maxpool2d('pool3', conv3)
    # 规范化
    norm3 = norm('norm3', pool3)
    
    # 第四层卷积
    conv4 = conv2d('conv4', norm3, weights['wc4'], biases['bc4'])
    
    # 第五层卷积
    conv5 = conv2d('conv5', conv4, weights['wc5'], biases['bc5'])
    # 下采样
    pool5 = maxpool2d('pool5', conv5)
    # 规范化
    norm5 = norm('norm5', pool5)
    
    # 全连接层
    fc1 = tf.reshape(norm5, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    fc1 = tf.nn.dropout(fc1, dropout)
    
    # 全连接层
    fc2 = tf.reshape(fc1, [-1, weights['wd2'].get_shape().as_list()[0]])
    fc2 = tf.add(tf.matmul(fc2, weights['wd2']), biases['bd2'])
    fc2 = tf.nn.relu(fc2)
    fc2 = tf.nn.dropout(fc2, dropout)
    
    out = tf.add(tf.matmul(fc2, weights['out']), biases['out'])
    return out

# 构建模型，定义损失函数和优化器，并构建评估函数
# 构建模型
pred = alex_net(x, weights, biases, dropout)

# 定义损失函数和优化器
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, y)) 
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost) 

# 评估函数 
correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1)) 
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))


# 训练模型和评估模型 
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    step = 1
    
    while step * batch_size < training_iters:
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        sess.run(optimizer, feed_dict={x: batch_x, y: batch_y, keep_prob: 1.})
        if step % display_step == 0:
            loss, accu = sess.run([cost, accu], feed_dict={x:batch_x, y:batch_y, keep_prob=1.})
            print("Iter " + str(step*batch_size) + ", Minibathc loss= " + \
                 "{:.6f}".format(loss) + ", Training Accuracy= " + \
                 "{:.5f}".format(accu))
        step += 1
        
print("Testing Accuracy:", \ 
    sess.run(accuracy, feed_dict={x: mnist.test.images[:256], 
    y: mnist.test.labels[:256], 
    keep_prob: 1.}))